<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/notebooks/DataSynthesisPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Synthesis

In [1]:
'''
TODO:
- TV query generation: (single query, chain of dependent queries, chain of independent query) -> list
- Generate creative full command from query (human-like conversation, more objects)
'''

'\nTODO:\n- TV query generation: (single query, chain of dependent queries, chain of independent query) -> list\n- Generate creative full command from query (human-like conversation, more objects)\n'

## TV Commands

In [2]:
tv_command_templates = [
    # Power control
    "turn on the TV",
    "turn off the TV",
    "restart the TV",
    "put the TV in sleep mode",

    # Volume control
    "increase the volume",
    "increase the volume to [level]",
    "decrease the volume",
    "decrease the volume to [level]",
    "set volume to [level]",
    "mute the TV",
    "unmute the TV",
    "set the volume to maximum",
    "set the volume to minimum",

    # Channel control
    "change the channel",
    "next channel",
    "previous channel",
    "go to channel [channel number]",
    "switch to channel [channel name]",
    "show me [channel name]",

    # App control
    "open [app name]",
    "launch [app name]",
    "close [app name]",
    "switch to [app name]",
    "search on [app name] for [query]",

    # Media playback
    "play",
    "pause",
    "resume",
    "stop",
    "rewind",
    "rewind [seconds] seconds",
    "fast forward",
    "fast forward [seconds] seconds",
    "skip intro",
    "skip to next episode",
    "go back to previous episode",

    # Subtitles and language
    "turn on subtitles",
    "turn off subtitles",
    "change subtitle language to [language]",
    "change audio language to [language]",

    # Input source
    "switch to [input source]",
    "change input to [input source]",
    "go to HDMI [number]",
    "switch to AV mode",

    # Picture and audio settings
    "set brightness to [level]",
    "set contrast to [level]",
    "enable night mode",
    "disable night mode",
    "enable game mode",
    "disable game mode",
    "set picture mode to [mode name]",
    "set sound mode to [mode name]",

    # Smart features
    "record this show",
    "show the TV guide",
    "what’s playing now",
    "show me recommendations",
    "add this to my watchlist",
    "rate this show [rating]",
    "enable sleep timer for [minutes] minutes",
    "remind me when [show name] starts",

    # Navigation
    "go back to home screen",
    "open settings",
    "scroll up",
    "scroll down",
    "select this option",
]

## Content list

In [27]:
# songs datasets
import pandas as pd
song_dataset = pd.read_csv("hf://datasets/vishnupriyavr/spotify-million-song-dataset/spotify_millsongdata.csv")

In [40]:
# movie datasets
import pandas as pd

movie_dataset = pd.read_csv("hf://datasets/Pablinho/movies-dataset/9000plus.csv")

In [118]:
app_dataset = [
    # 1. Video tổng hợp
    "YouTube",
    "TikTok TV",
    "Facebook Watch",
    "Twitch",
    "Dailymotion",
    "Vimeo",
    "Rumble",
    "Bilibili",
    "TED",
    "Vevo TV",

    # 2. Xem phim/truyền hình
    "Netflix",
    "Disney+",
    "Amazon Prime Video",
    "Apple TV+",
    "HBO Max",
    "Hulu",
    "Paramount+",
    "Peacock",
    "Crunchyroll",
    "Plex",
    "Tubi TV",
    "Pluto TV",
    "Rakuten TV",
    "Viki",
    "Popcornflix",

    # 3. Nghe nhạc và radio
    "Spotify",
    "Apple Music",
    "YouTube Music",
    "Amazon Music",
    "Tidal",
    "Deezer",
    "Pandora",
    "SoundCloud",
    "iHeartRadio",
    "TuneIn Radio",

    # 4. Truyền hình trực tiếp & thể thao
    "ESPN",
    "DAZN",
    "NBC Sports",
    "CBS Sports",
    "Red Bull TV",
    "BBC iPlayer",
    "ITVX",
    "Sling TV"
]

## Model and function

In [114]:
prompt_template = """
You are given the following:

- A list of TV control commands: {command_list}
- A content list that includes names of songs, TV shows, movies, apps, and artists: {content_list}

Your task is to generate {generated_nums} human-like commands using the command list and the content list. The content list includes the prefix movies or artist-song to shows you their type, you should remove them when generate commands, also leverage them smartly for high human-like commands.
Half of the commands should be **single commands**, and the other half should be **chain commands**. A chain command consists of multiple actions (up to a maximum of {chain_length} sub-commands) combined in a natural way.

Be creative in your generation. You should use specific content from the content list—such as names of songs, movies, TV shows, or artists—instead of generic phrases like "this song" or "that movie".

Each generated command should be as **realistic and unique** as possible.

Return only the generated human-like commands in the **following JSON format** (use **double quotes** for all strings, and ensure the output is valid JSON):

```json
{{
  "single": [
    "string"
  ],
  "chain": [
    {{
      "full_commands": "string",
      "sub_commands": ["string"]
    }}
  ]
}}
"""

GENERATED_NUM = 20
CHAIN_LENGTH = 3

In [115]:
from openai import OpenAI
import os
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key = api_key)

MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.8

In [126]:
import random

def content_sample(num_samples):
  num_songs = num_samples//2
  num_movies = num_samples - num_songs
  song_sample = song_dataset.sample(num_songs).reset_index(drop = True)
  movie_sample = movie_dataset.sample(num_movies).reset_index(drop = True)

  song_list = [f"artist: {song_sample.iloc[i]['artist']}- song: {song_sample.iloc[i]['song']}" for i in range(num_songs)]
  movie_list = [f"movie: {movie_sample.iloc[i]['Title']}" for i in range(num_movies)]
  app_list = [f"app: {app_dataset[i]}" for i in range(len(app_dataset))]
  return song_list, movie_list, app_list

def command_sample(num_samples):
  samples = random.sample(tv_command_templates, num_samples)
  return samples

def generate(command_list, content_list, generated_num = GENERATED_NUM, chain_length = CHAIN_LENGTH):
  prompt = prompt_template.format(command_list=command_list, content_list = content_list, generated_nums=generated_num, chain_length=chain_length)
  print(prompt)
  response = client.responses.create(
      model = MODEL,
      input = prompt,
      temperature = TEMPERATURE,
  )

  return response

In [127]:
command_list = command_sample(20)
song_list, movie_list, app_list = content_sample(20)

content_list = song_list + movie_list + app_list

In [128]:
response = generate(command_list, content_list)


You are given the following:

- A list of TV control commands: ['go to channel [channel number]', 'scroll down', 'switch to channel [channel name]', 'add this to my watchlist', 'set the volume to minimum', 'put the TV in sleep mode', 'skip intro', 'close [app name]', 'show me [channel name]', 'turn on the TV', 'disable game mode', 'disable night mode', 'search on [app name] for [query]', 'set volume to [level]', 'remind me when [show name] starts', 'change the channel', 'show the TV guide', 'scroll up', 'set the volume to maximum', 'mute the TV']
- A content list that includes names of songs, TV shows, movies, apps, and artists: ['artist: The Monkees- song: Gonna Buy Me A Dog', 'artist: Spandau Ballet- song: Toys', 'artist: Leann Rimes- song: Sure Thing', 'artist: Rammstein- song: Wut Will Nicht Sterben (Rage Does Not Want To Die)', "artist: Dean Martin- song: I'm Leaving It (all) Up To You", "artist: Celine Dion- song: It's All Coming Back To Me Now", 'artist: Israel Houghton- song: 

In [129]:
raw_output = response.output[0].content[0].text

import json
import re

def clean_json_string(raw_output):
    # Bỏ phần ```json và ```
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_output.strip())
    return cleaned

def parse_generated_json(raw_output):
    cleaned = clean_json_string(raw_output)
    return json.loads(cleaned)

# Example usage
parsed_data = parse_generated_json(raw_output)
parsed_data

{'single': ['go to channel 45',
  'show me YouTube',
  'set the volume to maximum',
  "search on Netflix for My Boyfriend's Meds",
  'add this to my watchlist',
  'mute the TV',
  'turn on the TV',
  'skip intro'],
 'chain': [{'full_commands': 'put the TV in sleep mode after watching The Rover',
   'sub_commands': ['show me The Rover', 'put the TV in sleep mode']},
  {'full_commands': 'scroll down and remind me when Gonna Buy Me A Dog starts',
   'sub_commands': ['scroll down',
    'remind me when Gonna Buy Me A Dog starts']},
  {'full_commands': 'switch to channel HBO Max and search for Cry',
   'sub_commands': ['switch to channel HBO Max', 'search on HBO Max for Cry']},
  {'full_commands': 'set the volume to minimum and disable game mode',
   'sub_commands': ['set the volume to minimum', 'disable game mode']}]}

In [74]:
dict

{'single': ["turn on the TV to watch 'Stranger Things'",
  "increase the volume for the latest episode of 'The Crown'",
  'switch to channel 7 for the news',
  'record this show so I can watch it later',
  'enable sleep timer for 30 minutes',
  "change audio language to Spanish for 'Money Heist'",
  'decrease the volume to 10 for late-night viewing',
  "resume playing 'The Mandalorian' from where I left off",
  "search on Netflix for 'The Witcher'",
  'put the TV in sleep mode after the movie ends'],
 'chain': [{'full_commands': 'increase the volume and switch to channel 5',
   'sub_commands': ['increase the volume', 'switch to channel 5']},
  {'full_commands': 'turn off the TV after the episode finishes',
   'sub_commands': ['turn off the TV']},
  {'full_commands': 'enable night mode and decrease the volume to 5',
   'sub_commands': ['enable night mode', 'decrease the volume to 5']},
  {'full_commands': "rewind 30 seconds and resume playing 'Game of Thrones'",
   'sub_commands': ['rew

In [76]:
content_list

['artist: Neil Diamond- song: New Orleans',
 'artist: Cher- song: I Walk Alone',
 'artist: Bing Crosby- song: And The Bells Rang',
 'artist: Reba Mcentire- song: I Know How He Feels',
 'artist: John McDermott- song: The Massacre Of Glencoe',
 'artist: Nicki Minaj- song: Hard Lemonade',
 'artist: Kim Wilde- song: Hypnotise',
 'artist: Regine Velasquez- song: Hotstuff',
 "artist: Christy Moore- song: Bridget's Pill",
 'artist: Bonnie Raitt- song: You Got To Know How',
 'movie: Compadres',
 'movie: Skins',
 'movie: Capernaum',
 'movie: The Last Castle',
 'movie: Free Willy 2: The Adventure Home',
 'movie: Days of Being Wild',
 'movie: The Lift',
 'movie: Unknown',
 'movie: Dragon Ball: The Magic Begins',
 'movie: Playing with Love']